Datasets:
- CSV 1: international_matches.csv

Columnas: ID, Tournament, Date, Home Team, Home Goals, Away Goals, Away Team, Win Conditions, Home Stadium
- CSV 2: 2022_world_cup_matches.csv
Columnas: ID, Year, Date, Stage, Home Team, Away Team, Host Team

En este ejercicio se pide combinar estos datos con información adicional de los continentes a los que pertenecen los equipos y luego asignar una probabilidad de victoria basada en los históricos.


Objetivos:
1. Cargar datos y explorarlos.
2. Limpieza y preparación de datos
3. Preparación de datos adicionales y merge
4. Cálculo de probabilidades de victoria bada en histórico
5. Análisis exploratorio.
6. Visualización.


## 1. Carga de los datos


In [ ]:
import pandas as pd

# 1. Cargar los archivos CSV
# Asegúrate de que los archivos estén en la misma carpeta que tu script o usa la ruta completa
df_matches = pd.read_csv('2022_world_cup_matches.csv')
df_groups = pd.read_csv('2022_world_cup_groups.csv')

# 2. Verificar estructura, tipos de datos y primeras filas del archivo de PARTIDOS
print("--- Dataframe Partidos (Matches) ---")
print("Primeras filas:")
print(df_matches.head())
print("\nInformación y Tipos de datos:")
print(df_matches.info())

# 3. Verificar estructura, tipos de datos y primeras filas del archivo de GRUPOS
print("\n--- Dataframe Grupos (Groups) ---")
print("Primeras filas:")
print(df_groups.head())
print("\nInformación y Tipos de datos:")
print(df_groups.info())

--- Dataframe Partidos (Matches) ---
Primeras filas:
   ID  Year        Date        Stage      Home Team    Away Team Host Team
0   1  2022  2022-11-20  Group stage          Qatar      Ecuador      True
1   2  2022  2022-11-21  Group stage        Senegal  Netherlands     False
2   3  2022  2022-11-21  Group stage        England         Iran     False
3   4  2022  2022-11-21  Group stage  United States        Wales     False
4   5  2022  2022-11-22  Group stage         France    Australia     False

Información y Tipos de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         64 non-null     int64 
 1   Year       64 non-null     int64 
 2   Date       64 non-null     object
 3   Stage      64 non-null     object
 4   Home Team  64 non-null     object
 5   Away Team  64 non-null     object
 6   Host Team  48 non-null     object
dtypes: int64

## 2. Limpieza y preparación de datos

In [ ]:
# Conteo de valores nulos por columna
print("Valores nulos por columna:")
print(df.isna().sum())

# Rellenar NaN en columnas numéricas clave con valores por defecto
df.fillna({'screen_time(min)': 0, 'notifications': 0}, inplace=True)

# Eliminar filas que no tengan app o tiempo de pantalla (esenciales para análisis)
df.dropna(subset=['app_name', 'screen_time(min)'], inplace=True)

print('\nDespués de limpieza de NaN:')
print(df.isna().sum())

display(df.head())


Valores nulos por columna:
user_id              0
app_name             0
screen_time(min)     2
notifications        2
date                 1
comentarios         17
dtype: int64

Después de limpieza de NaN:
user_id              0
app_name             0
screen_time(min)     0
notifications        0
date                 1
comentarios         17
dtype: int64


,user_id,app_name,screen_time(min),notifications,date,comentarios
0,1001,Instagram,45.0,10.0,2024-05-12,NaN
1,1002,WhatsApp,60.0,12.0,2024-05-12,NaN
2,1003,YouTube,120.0,8.0,2024-05-13,NaN
3,1004,Telegram,30.0,5.0,2024-05-14,NaN
4,1021,Instagram,0.0,14.0,2024-05-12,NaN


## 3. Preparación de datos adicionales y merge

In [8]:
# Detectar filas duplicadas completas
duplicados = df[df.duplicated()]
print("Filas duplicadas detectadas:")
display(duplicados)

# Eliminar duplicados
df.drop_duplicates(inplace=True)

print("Tamaño tras eliminar duplicados:", df.shape)


Filas duplicadas detectadas:


,user_id,app_name,screen_time(min),notifications,date,comentarios
7,1002,WhatsApp,60.0,12.0,2024-05-12,NaN


Tamaño tras eliminar duplicados: (16, 6)


## 4. Cálculo de probabilidades de victoria basada en histórico

In [ ]:
print("Tipos ANTES:")
print(df.dtypes)

# Convertir columnas numéricas que puedan venir como texto
df['screen_time(min)'] = df['screen_time(min)'].astype(float)

# notifications puede tener mezcla de enteros y strings -> forzamos conversión segura
df['notifications'] = pd.to_numeric(df['notifications'], errors='coerce').fillna(0).astype(int)

# Convertir fechas a datetime, forzando formato cuando hay mezcla
df['date'] = pd.to_datetime(df['date'], errors='coerce', dayfirst=True)

print("\nTipos DESPUÉS:")
print(df.dtypes)

display(df.head())


## 5. Análisis exploratorio

In [2]:
# Eliminamos columnas que estén completamente vacías
df.dropna(axis=1, how='all', inplace=True)

print("Columnas actuales del DataFrame:")
print(df.columns.tolist())


NameError: name 'df' is not defined

## 6. Visualización

In [ ]:
# Normalizar nombres de apps:
# - quitar espacios en blanco
# - poner la primera letra en mayúscula de cada palabra
# - eliminar espacios internos (Instagram  -> Instagram, 'YouTube Music' -> 'Youtubemusic')
df['app_name'] = (
    df['app_name']
      .astype(str)
      .str.strip()
      .str.title()
      .str.replace(' ', '', regex=False)
)

display(df[['app_name']].head(10))

# Buscar todas las filas que contienen "Youtube"
youtube_rows = df[df['app_name'].str.contains('Youtube', case=False, na=False)]
print("Filas con Youtube:")
display(youtube_rows)


In [ ]:
# Longitud del nombre de la app
df['app_name_len'] = df['app_name'].str.len()

# Apps que empiezan por 'Y'
apps_y = df[df['app_name'].str.startswith('Y', na=False)]

print("Apps que empiezan por 'Y':")
display(apps_y[['app_name', 'app_name_len']].drop_duplicates())

display(df[['app_name', 'app_name_len']].head())


In [ ]:
print("Tipos actuales:")
print(df.dtypes)

print("\nTipo real de cada celda en las primeras filas:")
display(df.applymap(type).head())

# Convertir automáticamente al tipo más adecuado (opcional)
df = df.convert_dtypes()

print("\nTipos tras convert_dtypes():")
print(df.dtypes)


In [ ]:
df.to_csv('screentime_analysis_clean.csv', index=False)
print('Archivo screentime_analysis_clean.csv guardado correctamente.')

In [3]:
# ¿Qué apps tienen más tiempo medio de pantalla?
screen_time_media = (
    df.groupby('app_name')['screen_time(min)']
      .mean()
      .sort_values(ascending=False)
)

print("Tiempo medio de pantalla (min) por app:")
display(screen_time_media)

# ¿Qué apps generan más notificaciones?
notif_media = (
    df.groupby('app_name')['notifications']
      .mean()
      .sort_values(ascending=False)
)

print("Notificaciones medias por app:")
display(notif_media)


Tiempo medio de pantalla (min) por app:


app_name
YouTube Music    130.0
YouTube          120.0
Netflix          110.0
Tik tok          100.0
YouTube Kids      95.0
  WhatsApp        80.0
WhatsApp          80.0
Telegram          60.0
Calendar Pro      50.0
Instagram         45.0
 instagram        30.0
TikTok             NaN
Name: screen_time(min), dtype: float64

Notificaciones medias por app:


app_name
  WhatsApp       15.0
Tik tok          12.0
Instagram        12.0
WhatsApp         12.0
YouTube Music     9.0
YouTube           8.0
YouTube Kids      7.0
Netflix           6.0
 instagram        6.0
Telegram          5.0
Calendar Pro      3.0
TikTok            NaN
Name: notifications, dtype: float64